In [43]:
import numpy as np
from torch.utils.data import DataLoader
from functions import *
from torch.nn import GRU,Conv1d,Linear,ReLU,Sequential,MSELoss,L1Loss,LeakyReLU,Dropout
from torch.optim import Adam
from pytorch_util import *
from pytorch_models import ConvBatchRelu1D

In [3]:
long_page = np.load('../Data/long_page.npy')
long_value = np.load('../Data/long_value.npy')
start_index = np.load('../Data/start_index.npy')
stds = np.load('../Data/stds_log.npy')
means = np.load('../Data/means_log.npy')
long_normal_value = np.load('../Data/long_lognormal_value.npy')

In [4]:
# to account for look back period 365 + 60
start_index = start_index + 430

In [5]:
## to change ##
batch_size = 64
clip = .5

In [6]:
# baseline model
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# SMAPE_np(y,yhat)
# >> 106.31
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# np.mean(np.square(y - yhat))
# >>> 1.1114173

In [9]:
# set up data loader
train_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,useMean=True)
train_gen = DataLoader(train_gen,batch_size,True,num_workers=3,drop_last=True)

val_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,random=False,useMean=True)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=3)

train_gen_nor = SequenceGenNormalized(long_normal_value,start_index)
train_gen_nor = DataLoader(train_gen_nor,batch_size,True,num_workers=3,drop_last=True)

val_gen_nor = SequenceGenNormalized(long_normal_value,start_index,random=False)
val_gen_nor = DataLoader(val_gen_nor,batch_size)

In [104]:
conv = Sequential(ConvBatchRelu1D(3,8,3),
                  ConvBatchRelu1D(8,16,3),
                  ConvBatchRelu1D(16,32,3),
                  ConvBatchRelu1D(32,64,3))

In [105]:
rnn = GRU(64,64,2,batch_first=True,dropout=0.5)

In [106]:
linear = Sequential(Linear(64,64),LeakyReLU(0.1,True),Dropout(0.5),Linear(64,1))

In [107]:
# model trained on normalized data
model_nor = CNN_RNN2seq_normalized(conv,rnn,linear).to('cuda:0')

In [108]:
loss_func_nor = loss_func_generator_normalized(MSELoss())
# loss_func_nor = loss_func_generator_normalized(L1Loss())
#loss_func_nor = loss_func_generator_normalized(SMAPE)

In [109]:
opt = Adam(trainable_parameter(model_nor),lr=1e-3)

In [110]:
model_nor = fit(3, model_nor, loss_func_nor, opt, train_gen_nor, val_gen_nor,clip)

epoch:0, train_loss:0.7897863072434855, val_loss:1.0042483806610107
epoch:1, train_loss:0.7552064328304844, val_loss:1.0226459503173828
epoch:2, train_loss:0.7481955734868124, val_loss:0.9557684659957886
Training completed in 85.16947817802429s


In [112]:
model = CNN_RNN2seq_logNormalized(conv,rnn,linear).to('cuda:0')

In [113]:
opt = Adam(trainable_parameter(model),lr=1e-4)

In [114]:
loss_func = loss_func_generator(SMAPE)

In [115]:
model = fit(5, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:37.040464486235784, val_loss:41.216087341308594
epoch:1, train_loss:36.97963347163225, val_loss:41.057167053222656
epoch:2, train_loss:36.90293212653442, val_loss:41.27570724487305
epoch:3, train_loss:36.821031866913636, val_loss:40.97426986694336
epoch:4, train_loss:36.76482305378494, val_loss:41.34160232543945
Training completed in 158.91707253456116s


In [116]:
model = fit(5, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:36.70716061270917, val_loss:40.9912109375
epoch:1, train_loss:36.722528153256434, val_loss:40.98737335205078
epoch:2, train_loss:36.71015673424913, val_loss:41.10382843017578
epoch:3, train_loss:36.66444116009332, val_loss:41.465660095214844
epoch:4, train_loss:36.66814113280934, val_loss:41.251522064208984
Training completed in 158.45669388771057s
